# Web Scraping Rental Apartment Data with Python and Beautifulsoup

Data from: https://www.immoscout24.ch/de/immobilien/mieten/ort-zuerich?pn=1

## Libraries and settings

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

## Example on how to scrape an element of the web page using BeautifulSoup

In [18]:
# Send a GET request to the input URL
response = requests.get('https://www.immoscout24.ch/de/immobilien/mieten/ort-zuerich?pn=1')

# Parse the content of the request with BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Extract the listings from the parsed content
# 'div' is the HTML tag (block-level container) of the element that contains the listings
# 'class_' is the CSS class of the element that contains the listings
listings = soup.find_all('div', class_='HgListingRoomsLivingSpacePrice_roomsLivingSpacePrice_M6Ktp')

# Show first 10 listings
listings[0:10]


[<div class="HgListingRoomsLivingSpacePrice_roomsLivingSpacePrice_M6Ktp"><strong>5.5 Zimmer</strong><span>, </span><strong title="Wohnfläche">140m²</strong><span class="HgListingRoomsLivingSpacePrice_commaPrice_mXXpt">,</span><span class="HgListingRoomsLivingSpacePrice_price_u9Vee"> CHF 5’950.– <!-- --><!-- --></span></div>,
 <div class="HgListingRoomsLivingSpacePrice_roomsLivingSpacePrice_M6Ktp"><strong>4.5 Zimmer</strong><span>, </span><strong title="Wohnfläche">89m²</strong><span class="HgListingRoomsLivingSpacePrice_commaPrice_mXXpt">,</span><span class="HgListingRoomsLivingSpacePrice_price_u9Vee"> CHF 4’680.– <!-- --><!-- --></span></div>,
 <div class="HgListingRoomsLivingSpacePrice_roomsLivingSpacePrice_M6Ktp"><strong>4.5 Zimmer</strong><span>, </span><strong title="Wohnfläche">88m²</strong><span class="HgListingRoomsLivingSpacePrice_commaPrice_mXXpt">,</span><span class="HgListingRoomsLivingSpacePrice_price_u9Vee"> CHF 4’590.– <!-- --><!-- --></span></div>,
 <div class="HgListin

## Web Scraper function

In [ ]:
# Function to extract the listings from a given URL
def extract_listing_info(url):

    # Send a GET request to the input URL
    response = requests.get(url)

    # Parse the content of the request with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract the listings from the parsed content
    listings = soup.find_all('div', class_='HgListingCard_info_RKrwz')
    
    # Initialize an empty list to store the extracted data
    extracted_data = []
    
    # Loop through the listings and extract the required information
    for listing in listings:
        # Extract address
        address_tag = listing.find('address')
        address = address_tag.text.strip() if address_tag else 'N/A'

        # Extract rooms, living area, and price
        info_section = listing.find('div', class_='HgListingRoomsLivingSpacePrice_roomsLivingSpacePrice_M6Ktp')
        if info_section:
            info_text = [strong.text.strip() for strong in info_section.find_all('strong')]
            rooms = info_text[0] if len(info_text) > 0 else 'N/A'
            living_area = info_text[1] if len(info_text) > 1 else 'N/A'
            price_tag = info_section.find('span', class_='HgListingRoomsLivingSpacePrice_price_u9Vee')
            price = price_tag.text.strip() if price_tag else 'N/A'
        else:
            rooms, living_area, price = 'N/A', 'N/A', 'N/A'

        # Extract title and description
        title_tag = listing.find('p', class_='HgListingDescription_title_NAAxy')
        title = title_tag.text.strip() if title_tag else 'N/A'

        description_tag = listing.find('p', class_='HgListingDescription_large_uKs3J')
        description = description_tag.text.strip() if description_tag else 'N/A'

        # Append extracted data to the list
        extracted_data.append({
            'address_raw': address,
            'rooms_raw': rooms,
            'area_raw': living_area,
            'price_raw': price,
            'title_raw': title,
            'description_raw': description
        })
    
    # Return the extracted data
    return extracted_data


## Call the Web Scraper function to get apartment data from the 1st page

In [ ]:
# Scrape 1st page
url = 'https://www.immoscout24.ch/de/immobilien/mieten/ort-zuerich?pn=1'

# Extract listing information to a list object 'data'
data = extract_listing_info(url)

# Create DataFrame from 'data'
df = pd.DataFrame(data)

# Show the first few rows of the DataFrame
df.head()

## Call the Web Scraper function to get apartment data from multiple pages

In [ ]:
# Scrape multiple pages
data = []
base_url = 'https://www.immoscout24.ch/de/immobilien/mieten/ort-zuerich?pn='

for page in range(1, 4):  # pages 1 to 3
    url = base_url + str(page)
    print(f"Scraping {url}...")
    data.extend(extract_listing_info(url))

# Create DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv('./data/rental_apartments_zuerich.csv', index=False)

print("\nData saved to real_estate_listings.csv")

# Show the first few rows of the DataFrame
df.head()

## Extract prices from strings using a regular expression (regex)

In [ ]:
# Extract prices from the 'price_raw' column using a regular expression (regex)
df['price'] = df['price_raw'].str.extract(r'(\d[\d’]*\d)')

# Remove thousands separator and convert to integer
df['price'] = df['price'].str.replace('’', '').astype(int)

# Show the first few rows of the DataFrame (selected columns)
df[['address_raw', 'rooms_raw', 'area_raw', 'price_raw', 'price']].head()

## Create histogram of prices

In [ ]:
# Plot a histogram of the prices
plt.figure(figsize=(7, 4))
plt.hist(df['price'], bins=30, color='greenyellow', edgecolor='gray')
plt.title('Histogram of prices')
plt.xlabel('Price (CHF)')
plt.ylabel('Frequency')
plt.show()


### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')